<a href="https://colab.research.google.com/github/kahramankaya/Anlas-l-rEkonomiYoutube/blob/main/81.video(One-Way%20MANOVA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#81.video(One-Way MANOVA):iki veya daha fazla bagımlı degısken oldugunda kullanılır. tek yonlu anova  tek bagımsız degıskende en az 3 grup olmalı. 1 tane bagımlı degısken de olmalı. ve bagımlı degısken sureklı olmalı.
# cıft yonlu anova da ıse yıne 1 tane bagımlı degısken olmalı ve sureklı olmalı. en az ıkı bagımsız degısken de olmalı.
#manova yapısında ise bagımlı degısken sayısı tek bır tane degıl. birden fazla ise ve de tek bır tane bagımsız degısken var ıse tek yonlu manova kullanılır.
#bagımlı degısken birden fazla iken, ve de 2 tane bagımsız degısken var ıse çift yonlu manova kullanılır.
#tek yonlu anova nın sartları: gozlemler bagımsız olmalı her zaman, her bagımlı degıskne sureklı ve bagımlı degıskenlerın normal dagılması lazım. varyans hoojenlıgı saglanmalıdır. var-cov matrısı esıtlıgıde olmalı.


In [21]:
import pandas as pd
import numpy as np
np.random.seed(5)
data=pd.DataFrame({
"erkek_tutumu":np.random.randint(1,25,size=(30)),
"kadın_tutumu":np.random.randint(1,25,size=(30)),
"urunler":np.random.choice(["A","B","C","D"],size=(30))
})
print(data)


    erkek_tutumu  kadın_tutumu urunler
0              4            10       A
1             15             2       B
2             16            19       D
3              7             8       D
4             23            17       D
5             17            15       C
6             10             6       D
7              9             1       B
8              5            17       B
9              8             5       B
10            17            15       B
11            17             5       D
12             8            10       A
13            13            20       C
14            16             3       C
15            18             5       C
16            22             7       D
17             8            10       B
18            17            20       C
19            13            20       D
20            14            19       A
21            12            18       B
22             2            22       C
23            16             8       B
24            21         

In [26]:
#1.adım normallıge bakılır.
#2.adım homojenlıge bakılır.
#3.adım  varcov'a bak.
!pip install pingouin
import pingouin as pg
erkekhomojenlık=pg.homoscedasticity(data,dv="erkek_tutumu",group="urunler",center="mean")
#erkek ve kadın ıcın ıcın homojenlık saglandı. o zaman var-cov matrısı esitliginden devam edebılırız. daha sonra da manova yapısına gecılır.
print(erkekhomojenlık)
kadınhomojenlık=pg.homoscedasticity(data,dv="kadın_tutumu",group="urunler",center="mean")
print(erkekhomojenlık)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
               W      pval  equal_var
levene  0.218642  0.882564       True
               W      pval  equal_var
levene  0.218642  0.882564       True


In [28]:
varcov=pg.box_m(data,dvs=["erkek_tutumu","kadın_tutumu"],group="urunler")
#h0 kabul edıldı.
print(varcov)

        Chi2   df      pval  equal_cov
box  3.39182  9.0  0.946718       True


In [34]:
from statsmodels.multivariate.manova import MANOVA
model=MANOVA.from_formula("erkek_tutumu+kadın_tutumu~urunler",data=data)
# altta wilks kısmından Pr > F degerı 0.7465 h0 kabul edecegız anlamı tasır.
print(model.mv_test())
#demek kı cınsıyetın urun tutumu uzerınde bır farkı yokmus.

                 Multivariate linear model
                                                            
------------------------------------------------------------
       Intercept        Value  Num DF  Den DF F Value Pr > F
------------------------------------------------------------
          Wilks' lambda 0.4080 2.0000 25.0000 18.1351 0.0000
         Pillai's trace 0.5920 2.0000 25.0000 18.1351 0.0000
 Hotelling-Lawley trace 1.4508 2.0000 25.0000 18.1351 0.0000
    Roy's greatest root 1.4508 2.0000 25.0000 18.1351 0.0000
------------------------------------------------------------
                                                            
------------------------------------------------------------
        urunler         Value  Num DF  Den DF F Value Pr > F
------------------------------------------------------------
          Wilks' lambda 0.8746 6.0000 50.0000  0.5774 0.7465
         Pillai's trace 0.1294 6.0000 52.0000  0.5993 0.7295
 Hotelling-Lawley trace 0.1388 6.0000 31.6

In [36]:
#dıyelim ki manova testınde h0 red edılırse o zaman surdan devam edılır:posthoc
posthoc1=pg.pairwise_tukey(data=data,dv="erkek_tutumu",between="urunler")
posthoc2=pg.pairwise_tukey(data=data,dv="kadın_tutumu",between="urunler")
print(posthoc1)
print(posthoc2)
# p tukey degerlerıne bak ve 0.05 ıle karsılastır.buyukse farklılık yok tersı ıse farklılık var demek

   A  B    mean(A)    mean(B)      diff        se         T   p-tukey  \
0  A  B  11.800000  12.111111 -0.311111  3.195450 -0.097361  0.999660   
1  A  C  11.800000  13.285714 -1.485714  3.354524 -0.442899  0.970404   
2  A  D  11.800000  15.555556 -3.755556  3.195450 -1.175282  0.647364   
3  B  C  12.111111  13.285714 -1.174603  2.887115 -0.406843  0.976770   
4  B  D  12.111111  15.555556 -3.444444  2.700648 -1.275414  0.586175   
5  C  D  13.285714  15.555556 -2.269841  2.887115 -0.786197  0.860027   

     hedges  
0 -0.050593  
1 -0.215705  
2 -0.562371  
3 -0.211958  
4 -0.619174  
5 -0.376170  
   A  B    mean(A)    mean(B)      diff        se         T   p-tukey  \
0  A  B  11.200000  10.000000  1.200000  3.617526  0.331718  0.987118   
1  A  C  11.200000  13.857143 -2.657143  3.797611 -0.699688  0.896257   
2  A  D  11.200000  11.000000  0.200000  3.617526  0.055286  0.999938   
3  B  C  10.000000  13.857143 -3.857143  3.268464 -1.180109  0.644429   
4  B  D  10.000000  11.00